In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

## Profile Clustering

In [ ]:
df = pd.read_excel("...Profile_Data\Github_Profiles_Filtered.xlsx")
df

,username,name,email,location,public_repos,followers,following,total_stars,total_forks,top_languages,common_topics
0,mangleirl,NaN,NaN,graphics r n͟o͟t f2u . ㅤㅤㅤㅤㅤㅤㅤㅤㅤㅤpfp cr : ange...,1,707,6214,2,0,NaN,NaN
1,devridge0,DevRidge,NaN,NaN,43,956,2003,328,7,TypeScript; Python; Rust; JavaScript; Solidity,version-control; decentralized-finance; crytoc...
2,FellouAI,FellouAI,NaN,United States of America,8,606,0,4953,482,TypeScript; HTML; MDX; Ruby; JavaScript,workflow; agent; agentic-ai-development; agent...
3,spider0317,Dream comes true!,NaN,NaN,13,960,971,23,14,JavaScript; Solidity; Python; TypeScript,NaN
4,HandsOnLLM,NaN,NaN,NaN,2,736,0,20101,4788,Jupyter Notebook,llm; large-language-models; llms; oreilly-book...
...,...,...,...,...,...,...,...,...,...,...,...
987,DevNinja08,Dev Ninja,NaN,NaN,13,2928,6474,92,6,JavaScript,NaN
988,Ahmad-2785,NaN,NaN,NaN,115,603,889,130,13,Shell,NaN
989,CheshireCatful,꣑୧ 𝒴oko / Mangle ꒰ 🦭,NaN,︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵ used to be MissConstruction,3,539,498,3,0,NaN,NaN
990,slycompiler,Senior Smart Contract Engineer | Rust & Solidi...,NaN,NaN,80,653,8,158,41,Rust; TypeScript; JavaScript; Solidity; Go,solidity; initialcoinoffering; nearprotocol; g...


In [ ]:
# Download model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Clean Data
df['top_languages'] = df['top_languages'].fillna('').astype(str)
df['common_topics'] = df['common_topics'].fillna('').astype(str)

# Combine columns
df['combined_info'] = df['top_languages'] + " " + df['common_topics']

# Define and Encode target roles
TARGET_ROLES = [
    "Data Scientist", "Data Engineer", "Solutions Architect",
    "Business Analyst", "Full Stack Developer", "Mobile Developer",
    "DevOps Engineer", "Backend Developer", "Frontend Developer",
    "Blockchain Developer", "ML Engineer", "Security Engineer"
]
role_embeddings = model.encode(TARGET_ROLES, convert_to_tensor=True)

# Process all candidates together
print("Classifying candidates...")
candidate_embeddings = model.encode(df['combined_info'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Calculate Similarity (with cos_sim)
cosine_scores = util.cos_sim(candidate_embeddings, role_embeddings)

# Find the highest scoring role for each candidate
best_role_indices = torch.argmax(cosine_scores, dim=1)
df['developer_profile'] = [TARGET_ROLES[idx] for idx in best_role_indices]

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Classifying candidates...


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Done!


In [43]:
df.to_csv('Profiles_Identified.csv')